## Model Deployment with TensorFlow Serving - Kubernetes

### Dockerize TensorFlow Serving

Create Dockerfile file:

```
# Use the official TensorFlow Serving image
FROM tensorflow/serving:latest

# Create the model directory at /models
RUN mkdir -p /models/saved_model/1

# Copy the model to the correct directory
COPY ./saved_model/* /models/saved_model/1/

# Set environment variable for TensorFlow Serving to use the model
ENV MODEL_NAME=saved_model

```

Build Docker image:

```
docker build -t e-commerce-engagement-model .
docker run -p 8501:8501 --name e-commerce-engagement e-commerce-engagement-model

```



## Container Orchestration wth Kubernetes

Create config files:
- deployment.yaml
- service.yaml


### deployment.yaml

```
apiVersion: apps/v1
kind: Deployment
metadata:
  name: e-commerce-engagement-model
spec:
  replicas: 2
  selector:
    matchLabels:
      app: e-commerce-engagement-model  # Make sure this matches the template labels
  template:
    metadata:
      labels:
        app: e-commerce-engagement-model
    spec:
      containers:
      - name: e-commerce-engagement-model
        image: e-commerce-engagement-model:latest 
        imagePullPolicy: IfNotPresent  # to solve READY 0/1 status (error) of kubectl get pods command 
        ports:
        - containerPort: 8501
# Note: apply with command: kubectl apply -f deployment.yaml

```

### service.yaml

```
apiVersion: v1
kind: Service
metadata:
  name: e-commerce-engagement-model
spec:
  type: NodePort 
  selector:
    app: e-commerce-engagement-model
  ports:
    - port: 80
      targetPort: 8501
      nodePort: 30007  # NodePort range: 30000-32767
  externalIPs: 
    - 192.168.68.100

```

In [1]:
!kubectl apply -f deployment.yaml

deployment.apps/e-commerce-engagement-model unchanged


In [2]:
!kubectl apply -f service.yaml

service/e-commerce-engagement-model unchanged


In [3]:
!kind load docker-image e-commerce-engagement-model:latest

Image: "e-commerce-engagement-model:latest" with ID "sha256:c2d8d64ef541518e54dd0baf0f615b330398160700bce824aae9fb503f2df16a" found to be already present on all nodes.


In [4]:
!kubectl get nodes

NAME                 STATUS   ROLES           AGE     VERSION
kind-control-plane   Ready    control-plane   6h59m   v1.25.3


In [5]:
!kubectl get services

NAME                          TYPE        CLUSTER-IP     EXTERNAL-IP      PORT(S)        AGE
e-commerce-engagement-model   NodePort    10.96.110.39   192.168.68.100   80:30007/TCP   110s
kubernetes                    ClusterIP   10.96.0.1      192.168.68.99    443/TCP        6h59m


In [6]:
# Retrieve information about all Kubernetes resources
!kubectl get all

NAME                                             READY   STATUS    RESTARTS   AGE
pod/e-commerce-engagement-model-85b4847d-dzczw   1/1     Running   0          6h51m
pod/e-commerce-engagement-model-85b4847d-rvgkm   1/1     Running   0          6h51m

NAME                                  TYPE        CLUSTER-IP     EXTERNAL-IP      PORT(S)        AGE
service/e-commerce-engagement-model   NodePort    10.96.110.39   192.168.68.100   80:30007/TCP   110s
service/kubernetes                    ClusterIP   10.96.0.1      192.168.68.99    443/TCP        6h59m

NAME                                          READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/e-commerce-engagement-model   2/2     2            2           6h51m

NAME                                                   DESIRED   CURRENT   READY   AGE
replicaset.apps/e-commerce-engagement-model-85b4847d   2         2         2       6h51m


In [7]:
!kubectl cluster-info

Kubernetes control plane is running at https://127.0.0.1:38847
CoreDNS is running at https://127.0.0.1:38847/api/v1/namespaces/kube-system/services/kube-dns:dns/proxy

To further debug and diagnose cluster problems, use 'kubectl cluster-info dump'.


In [8]:
!kubectl get nodes -o wide

NAME                 STATUS   ROLES           AGE     VERSION   INTERNAL-IP   EXTERNAL-IP   OS-IMAGE             KERNEL-VERSION       CONTAINER-RUNTIME
kind-control-plane   Ready    control-plane   6h59m   v1.25.3   172.18.0.2    <none>        Ubuntu 22.04.1 LTS   5.15.0-128-generic   containerd://1.6.9


In [9]:
# terminal
# !kubectl port-forward pod/e-commerce-engagement-model-85b4847d-dzczw 5000:5000

## Testing locally (noy using External IP)

In [10]:
!curl -X POST http://localhost:5000/predict -H "Content-Type: application/json" -d '{"text": "Hadiah langsung"}' 

{"prediction":[[23.862144470214844]]}


## External IP 

In [11]:
!kubectl get services

NAME                          TYPE        CLUSTER-IP     EXTERNAL-IP      PORT(S)        AGE
e-commerce-engagement-model   NodePort    10.96.110.39   192.168.68.100   80:30007/TCP   111s
kubernetes                    ClusterIP   10.96.0.1      192.168.68.99    443/TCP        6h59m


In [12]:
!kubectl get all

NAME                                             READY   STATUS    RESTARTS   AGE
pod/e-commerce-engagement-model-85b4847d-dzczw   1/1     Running   0          6h51m
pod/e-commerce-engagement-model-85b4847d-rvgkm   1/1     Running   0          6h51m

NAME                                  TYPE        CLUSTER-IP     EXTERNAL-IP      PORT(S)        AGE
service/e-commerce-engagement-model   NodePort    10.96.110.39   192.168.68.100   80:30007/TCP   111s
service/kubernetes                    ClusterIP   10.96.0.1      192.168.68.99    443/TCP        6h59m

NAME                                          READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/e-commerce-engagement-model   2/2     2            2           6h51m

NAME                                                   DESIRED   CURRENT   READY   AGE
replicaset.apps/e-commerce-engagement-model-85b4847d   2         2         2       6h51m


In [13]:
!kubectl describe service e-commerce-engagement-model


Name:                     e-commerce-engagement-model
Namespace:                default
Labels:                   <none>
Annotations:              <none>
Selector:                 app=e-commerce-engagement-model
Type:                     NodePort
IP Family Policy:         SingleStack
IP Families:              IPv4
IP:                       10.96.110.39
IPs:                      10.96.110.39
External IPs:             192.168.68.100
Port:                     <unset>  80/TCP
TargetPort:               8501/TCP
NodePort:                 <unset>  30007/TCP
Endpoints:                10.244.0.7:8501,10.244.0.8:8501
Session Affinity:         None
External Traffic Policy:  Cluster
Internal Traffic Policy:  Cluster
Events:                   <none>


In [14]:
!curl -X POST http://192.168.68.100:30007/predict -H "Content-Type: application/json" -d '{"text": "Hadiah langsung"}'


curl: (7) Failed to connect to 192.168.68.100 port 30007 after 2137 ms: Connection refused
